## R ve Tidy veri prensiplerini kullanarak K-Means kümeleme keşfi.

### [**Ders öncesi quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

Bu derste, Tidymodels paketi ve R ekosistemindeki diğer paketleri (onlara arkadaşlar 🧑‍🤝‍🧑 diyeceğiz) kullanarak kümeler oluşturmayı ve daha önce içe aktardığınız Nijerya müzik veri setini nasıl kullanacağınızı öğreneceksiniz. K-Means kümeleme için temel bilgileri ele alacağız. Unutmayın, önceki derste öğrendiğiniz gibi, kümelerle çalışmanın birçok yolu vardır ve kullandığınız yöntem verinize bağlıdır. En yaygın kümeleme tekniği olduğu için K-Means yöntemini deneyeceğiz. Haydi başlayalım!

Öğreneceğiniz terimler:

-   Siluet skoru

-   Dirsek yöntemi

-   Atalet

-   Varyans

### **Giriş**

[K-Means Kümeleme](https://wikipedia.org/wiki/K-means_clustering), sinyal işleme alanından türetilmiş bir yöntemdir. Verileri özelliklerindeki benzerliklere göre `k kümeye` ayırmak ve bölmek için kullanılır.

Kümeler, bir nokta (veya 'tohum') ve ona karşılık gelen bölgeyi içeren [Voronoi diyagramları](https://wikipedia.org/wiki/Voronoi_diagram) olarak görselleştirilebilir.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Jen Looper tarafından hazırlanan infografik</figcaption>

K-Means kümeleme şu adımları içerir:

1. Veri bilimci, oluşturulacak kümelerin istenen sayısını belirleyerek başlar.

2. Daha sonra algoritma, veri setinden rastgele K gözlem seçerek kümeler için başlangıç merkezleri (yani, merkez noktalar) olarak kullanır.

3. Ardından, kalan her bir gözlem en yakın merkez noktaya atanır.

4. Daha sonra, her bir kümenin yeni ortalamaları hesaplanır ve merkez noktası bu ortalamaya taşınır.

5. Merkezler yeniden hesaplandığına göre, her bir gözlem tekrar kontrol edilir ve farklı bir kümeye daha yakın olup olmadığına bakılır. Tüm nesneler, güncellenmiş küme ortalamalarını kullanarak yeniden atanır. Küme atama ve merkez noktası güncelleme adımları, küme atamaları değişmeyi bırakana kadar (yani, yakınsama sağlandığında) tekrarlanır. Genellikle, algoritma her yeni iterasyonda merkez noktalarının hareketi önemsiz hale geldiğinde ve kümeler statik hale geldiğinde sona erer.

<div>

> Başlangıç merkez noktaları olarak kullanılan ilk k gözlemlerin rastgele seçilmesi nedeniyle, prosedürü her uyguladığımızda biraz farklı sonuçlar alabiliriz. Bu nedenle, çoğu algoritma birkaç *rastgele başlangıç* kullanır ve en düşük WCSS'ye sahip iterasyonu seçer. Bu nedenle, K-Means'i her zaman birkaç *nstart* değeriyle çalıştırmanız ve *istenmeyen yerel optimumdan* kaçınmanız şiddetle tavsiye edilir.

</div>

Allison Horst'un [çizimleri](https://github.com/allisonhorst/stats-illustrations) kullanılarak hazırlanan bu kısa animasyon, kümeleme sürecini açıklıyor:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>@allison_horst tarafından hazırlanan çizim</figcaption>

Kümeleme ile ilgili temel bir soru şudur: Verilerinizi kaç kümeye ayırmanız gerektiğini nasıl bileceksiniz? K-Means kullanmanın bir dezavantajı, `k` yani `merkez noktalarının` sayısını belirlemeniz gerekmesidir. Neyse ki, `dirsek yöntemi` `k` için iyi bir başlangıç değeri tahmin etmenize yardımcı olur. Birazdan bunu deneyeceksiniz.

### 

**Ön Koşul**

[Önceki derste](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb) kaldığımız yerden devam edeceğiz. Bu derste veri setini analiz ettik, birçok görselleştirme yaptık ve ilgi çekici gözlemleri filtreledik. Mutlaka göz atın!

Bu modülü tamamlamak için bazı paketlere ihtiyacımız olacak. Şu şekilde yükleyebilirsiniz: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternatif olarak, aşağıdaki script, bu modülü tamamlamak için gerekli paketlere sahip olup olmadığınızı kontrol eder ve eksik olanları sizin için yükler.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Haydi başlayalım!

## 1. Verilerle dans: En popüler 3 müzik türünü belirleyin

Bu, önceki derste yaptıklarımızın bir özeti. Hadi biraz veri inceleyip analiz edelim!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Bu harika oldu!

## 2. Daha fazla veri keşfi.

Bu veriler ne kadar temiz? Aykırı değerleri kutu grafikleri kullanarak kontrol edelim. Daha az aykırı değere sahip sayısal sütunlara odaklanacağız (ancak aykırı değerleri temizleyebilirsiniz). Kutu grafikleri, verilerin aralığını gösterebilir ve hangi sütunların kullanılacağına karar vermenize yardımcı olabilir. Ancak unutmayın, kutu grafikleri varyansı göstermez, bu da iyi kümeleme yapılabilir veriler için önemli bir unsurdur. Daha fazla bilgi için lütfen [bu tartışmaya](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) göz atın.

[Kutu grafikleri](https://en.wikipedia.org/wiki/Box_plot), `sayısal` verilerin dağılımını görsel olarak göstermek için kullanılır, bu yüzden popüler müzik türleriyle birlikte tüm sayısal sütunları *seçerek* başlayalım.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Seçim yardımcısı `where`'in bunu ne kadar kolaylaştırdığını görüyor musunuz 💁? Bu tür diğer fonksiyonları [burada](https://tidyselect.r-lib.org/) keşfedin.

Her bir sayısal özellik için bir kutu grafiği oluşturacağımız ve döngü kullanmaktan kaçınmak istediğimiz için, verilerimizi *daha uzun* bir formata dönüştürelim. Bu, `facets`'ten - her biri verinin bir alt kümesini gösteren alt grafiklerden - faydalanmamıza olanak tanıyacak.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Daha uzun! Şimdi biraz `ggplot` zamanı! Peki hangi `geom`'u kullanacağız?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Kolay-gg!

Şimdi bu verilerin biraz gürültülü olduğunu görebiliyoruz: Her sütunu bir kutu grafiği olarak gözlemlediğinizde, aykırı değerleri fark edebilirsiniz. Bu aykırı değerleri veri setinden çıkarabilirsiniz, ancak bu durumda veri oldukça sınırlı hale gelir.

Şimdilik, kümeleme çalışmamızda kullanacağımız sütunları seçelim. Benzer aralıklara sahip sayısal sütunları seçelim. `artist_top_genre` sütununu sayısal olarak kodlayabiliriz, ancak şimdilik bunu çıkaracağız.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. R'de k-means kümeleme hesaplama

R'de yerleşik `kmeans` fonksiyonunu kullanarak k-means hesaplayabiliriz, bkz. `help("kmeans()")`. `kmeans()` fonksiyonu, birincil argüman olarak tüm sütunları sayısal olan bir veri çerçevesini kabul eder.

K-means kümeleme kullanırken ilk adım, nihai çözümde oluşturulacak küme sayısını (k) belirtmektir. Veri setinden çıkardığımız 3 şarkı türü olduğunu biliyoruz, o halde 3 deneyelim:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


kmeans nesnesi, `help("kmeans()")` içinde iyi bir şekilde açıklanan birkaç bilgi içerir. Şimdilik, birkaçına odaklanalım. Verilerin 65, 110, 111 boyutlarında 3 kümeye ayrıldığını görüyoruz. Çıktı ayrıca 5 değişken üzerinden 3 grup için küme merkezlerini (ortalama değerlerini) içerir.

Kümeleme vektörü, her bir gözlem için küme atamasını ifade eder. Küme atamasını orijinal veri setine eklemek için `augment` fonksiyonunu kullanalım.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Harika, veri setimizi 3 gruba ayırdık. Peki, kümelememiz ne kadar iyi 🤷? Hadi `Silhouette skoru`na bir göz atalım.

### **Silhouette skoru**

[Silhouette analizi](https://en.wikipedia.org/wiki/Silhouette_(clustering)), ortaya çıkan kümeler arasındaki ayrım mesafesini incelemek için kullanılabilir. Bu skor -1 ile 1 arasında değişir ve skor 1'e yakınsa, küme yoğun ve diğer kümelerden iyi bir şekilde ayrılmıştır. 0'a yakın bir değer, komşu kümelerin karar sınırına çok yakın örneklerle örtüşen kümeleri temsil eder. [kaynak](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Ortalama silhouette yöntemi, farklı *k* değerleri için gözlemlerin ortalama silhouette skorunu hesaplar. Yüksek bir ortalama silhouette skoru, iyi bir kümelemeyi gösterir.

`silhouette` fonksiyonu, cluster paketinde ortalama silhouette genişliğini hesaplamak için kullanılır.

> Silhouette, [mesafe](https://en.wikipedia.org/wiki/Distance "Distance") metriği ile hesaplanabilir, örneğin [Öklid mesafesi](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") veya [Manhattan mesafesi](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") gibi. Bu metrikleri [önceki derste](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb) tartışmıştık.


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Skorumuz **.549**, yani tam ortada. Bu, verilerimizin bu tür bir kümeleme için özellikle uygun olmadığını gösteriyor. Bu tahminimizi görsel olarak doğrulayıp doğrulayamayacağımıza bakalım. [factoextra paketi](https://rpkgs.datanovia.com/factoextra/index.html), kümelemeyi görselleştirmek için (`fviz_cluster()`) işlevlerini sağlar.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Kümeler arasındaki örtüşme, verilerimizin bu tür bir kümeleme için pek uygun olmadığını gösteriyor, ancak devam edelim.

## 4. Optimum küme sayısını belirleme

K-Means kümeleme ile ilgili sıkça ortaya çıkan temel bir soru şudur: Bilinen sınıf etiketleri olmadan, verilerinizi kaç kümeye ayırmanız gerektiğini nasıl bilebilirsiniz?

Bunu öğrenmenin bir yolu, bir veri örneği kullanarak `artan küme sayısıyla bir dizi kümeleme modeli oluşturmak` (örneğin 1'den 10'a kadar) ve **Silhouette skoru** gibi kümeleme metriklerini değerlendirmektir.

Optimum küme sayısını belirlemek için farklı *k* değerleri için kümeleme algoritmasını çalıştırıp **Küme İçi Kareler Toplamı** (WCSS) değerini değerlendirelim. Toplam küme içi kareler toplamı (WCSS), kümeleme kompaktlığını ölçer ve mümkün olduğunca küçük olmasını isteriz; daha düşük değerler, veri noktalarının birbirine daha yakın olduğunu gösterir.

Bu kümeleme üzerinde `k` için 1'den 10'a kadar farklı seçimlerin etkisini inceleyelim.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Her bir kümeleme algoritması için merkez *k* ile toplam küme içi kareler toplamını (tot.withinss) elde ettikten sonra, optimal küme sayısını bulmak için [dirsek yöntemi](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) kullanılır. Bu yöntem, WCSS'yi küme sayısının bir fonksiyonu olarak çizmek ve kullanılacak küme sayısı olarak [eğrinin dirseğini](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Eğrinin dirseği") seçmekten oluşur.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Grafik, kümelerin sayısı birden ikiye çıkarken WCSS'de (dolayısıyla daha fazla *sıkılık*) büyük bir azalma ve iki kümeden üç kümeye geçerken daha belirgin bir azalma gösteriyor. Bundan sonra azalma daha az belirgin hale geliyor ve grafikte yaklaşık üç kümede bir `dirsek` 💪 oluşuyor. Bu, veri noktalarının iki ila üç makul şekilde ayrılmış küme oluşturduğuna dair iyi bir göstergedir.

Şimdi `k = 3` olduğu durumda kümeleme modelini çıkarabiliriz:

> `pull()`: tek bir sütunu çıkarmak için kullanılır
>
> `pluck()`: listeler gibi veri yapılarında indeksleme yapmak için kullanılır


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Harika! Hadi elde edilen kümeleri görselleştirelim. `plotly` kullanarak biraz etkileşim eklemeye ne dersiniz?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Belki de her kümenin (farklı renklerle temsil edilen) farklı türlere (farklı şekillerle temsil edilen) sahip olmasını beklerdik.

Şimdi modelin doğruluğuna bir göz atalım.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Bu modelin doğruluğu fena değil, ama harika da değil. Bunun nedeni, verilerin K-Means Kümeleme için uygun olmaması olabilir. Bu veri seti çok dengesiz, çok az ilişkilendirilmiş ve sütun değerleri arasında çok fazla varyans var, bu da iyi bir kümeleme yapılmasını zorlaştırıyor. Aslında, oluşan kümeler muhtemelen yukarıda tanımladığımız üç tür kategorisinden büyük ölçüde etkileniyor veya çarpıtılıyor.

Yine de, bu oldukça öğretici bir süreçti!

Scikit-learn dokümantasyonunda, bu tür kümelerin çok iyi ayrılmadığı bir modelin 'varyans' problemi yaşadığını görebilirsiniz:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Scikit-learn'den bir bilgi grafiği</figcaption>



## **Varyans**

Varyans, "ortalamanın karesel farklarının ortalaması" olarak tanımlanır [kaynak](https://www.mathsisfun.com/data/standard-deviation.html). Bu kümeleme problemi bağlamında, veri setimizdeki sayıların ortalamadan biraz fazla sapma eğiliminde olduğunu ifade eder.

✅ Bu sorunu düzeltmek için düşünebileceğiniz tüm yolları değerlendirmek için harika bir an. Verileri biraz daha düzenlemek mi? Farklı sütunlar kullanmak mı? Farklı bir algoritma mı denemek? İpucu: Verilerinizi normalize etmek için [verilerinizi ölçeklendirmeyi deneyin](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) ve diğer sütunları test edin.

> Bu '[varyans hesaplayıcıyı](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' kullanarak kavramı biraz daha iyi anlayabilirsiniz.

------------------------------------------------------------------------

## **🚀Meydan Okuma**

Bu not defteriyle biraz zaman geçirin ve parametreleri değiştirin. Verileri daha fazla temizleyerek (örneğin, aykırı değerleri kaldırarak) modelin doğruluğunu artırabilir misiniz? Belirli veri örneklerine daha fazla ağırlık vermek için ağırlıklar kullanabilirsiniz. Daha iyi kümeler oluşturmak için başka neler yapabilirsiniz?

İpucu: Verilerinizi ölçeklendirmeyi deneyin. Not defterinde, veri sütunlarının aralık açısından birbirine daha çok benzemesini sağlamak için standart ölçeklendirme ekleyen yorumlanmış kodlar var. Verilerin ölçeklendirilmemiş haliyle bırakılması, daha az varyansa sahip verilerin daha fazla ağırlık taşımasına izin verdiği için, silüet puanı düşse de dirsek grafiğindeki 'kırılma' yumuşar. Bu sorun hakkında biraz daha fazla bilgi edinmek için [burayı okuyun](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Ders Sonrası Testi**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Gözden Geçirme ve Kendi Kendine Çalışma**

-   Bir K-Means Simülatörüne göz atın [örneğin bu](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Bu aracı kullanarak örnek veri noktalarını görselleştirebilir ve merkezlerini belirleyebilirsiniz. Verinin rastgeleliğini, küme sayılarını ve merkez sayılarını düzenleyebilirsiniz. Bu, verilerin nasıl gruplanabileceği hakkında bir fikir edinmenize yardımcı oluyor mu?

-   Ayrıca, Stanford'dan [bu K-Means el kitabına](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) göz atın.

Yeni edindiğiniz kümeleme becerilerinizi K-Means kümeleme için uygun veri setlerinde denemek ister misiniz? Şunlara göz atabilirsiniz:

-   [Kümeleme Modellerini Eğit ve Değerlendir](https://rpubs.com/eR_ic/clustering) Tidymodels ve arkadaşlarıyla

-   [K-means Kümeleme Analizi](https://uc-r.github.io/kmeans_clustering), UC İş Analitiği R Programlama Rehberi

- [Tidy veri ilkeleriyle K-means kümeleme](https://www.tidymodels.org/learn/statistics/k-means/)

## **Ödev**

[Farklı kümeleme yöntemlerini deneyin](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## TEŞEKKÜRLER:

[Jen Looper](https://www.twitter.com/jenlooper) bu modülün orijinal Python versiyonunu oluşturduğu için ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) R'ı daha sıcak ve çekici hale getiren harika illüstrasyonları için. Daha fazla illüstrasyonu [galerisinde](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM) bulabilirsiniz.

Keyifli Öğrenmeler,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Öğrenci Elçisi.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>@allison_horst tarafından yapılmış bir çalışma</figcaption>



---

**Feragatname**:  
Bu belge, [Co-op Translator](https://github.com/Azure/co-op-translator) adlı yapay zeka çeviri hizmeti kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
